In [ ]:
dt.itrial(20).attrs

In [9]:
"""Calculate per-frame motion score from video using frame differencing."""


import numpy as np

import subprocess
import json

def bitrate_motion(video_path: str) -> tuple[np.ndarray, np.ndarray]:
    cmd = [
        "ffprobe", "-select_streams", "v", "-show_frames",
        "-show_entries", "frame=pts_time,pkt_size",
        "-of", "json", video_path
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    frames = json.loads(result.stdout)["frames"]
    
    timestamps = np.array([float(f["pts_time"]) for f in frames])
    sizes = np.array([int(f["pkt_size"]) for f in frames])
    return timestamps, sizes


path = r"C:\Users\Admin\Downloads\video.mp4"
idx, scores = bitrate_motion(path)

## Modify AllTrials.nc

In [ ]:
%load_ext autoreload
%autoreload 2


from scipy.io import loadmat
import glob
import os
import json
import warnings
import numpy as np
from pathlib import Path
import xarray as xr
from ethograph.utils.io_matlab import update_dt_with_matlab_pulse_onsets
from ethograph.features.movement import compute_aux_velocity_and_speed
from ethograph.features.preprocessing import resample_to_frames, clip_by_percentiles, downsample_with_antialiasing
from ethograph import TrialTree


user = "Alice"
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
json_file = os.path.join(desktop_path, "user_paths.json")
with open(json_file, "r") as file:
    paths = json.load(file)
raw_videos_folder = paths[user]["raw_videos_folder"]


fps = 200

# raw!
behav_folders = [
                r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250527_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250527_02\behav",
                r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250528_01\behav",            
                r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250526_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250526_02\behav",
                r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250528_02\behav",
                r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250529_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250530_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-03_id-Freddy\ses-000_date-20250602_01\behav",
                
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250306_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250307_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250308_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250309_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250503_02\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250504_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250505_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250506_02\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250507_02\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250507_03\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250508_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250508_02\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250509_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250512_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250513_01\behav",
                
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250514_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250515_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250516_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250519_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250521_01\behav",
                r"D:\Alice\AK_data\rawdata\sub-01_id-Ivy\ses-000_date-20250522_01\behav"       
                ]

accel_path = r"D:\Alice\AK_data\rawdata\accelerometer_copied"

def read_intan_auxiliary(filepath: str, n_channels: int = 3) -> np.ndarray:
    """Read Intan auxiliary.dat file. Returns array in volts."""
    raw = np.fromfile(filepath, dtype=np.uint16)
    return raw.reshape(-1, n_channels) * 0.0000374

def read_intan_timestamps(filepath: str, sample_rate: float = 30000.0) -> np.ndarray:
    raw = np.fromfile(filepath, dtype=np.int32)
    return raw / sample_rate

for behav_folder in behav_folders:
    print(f"Processing folder: {behav_folder}")
    
    deriv_behav_folder = behav_folder.replace("rawdata", "derivatives")
    nc_path = Path(deriv_behav_folder) / "Trial_data.nc"
    
    dt = TrialTree.open(nc_path)
    
    

    AllTrialsPath = os.path.join(deriv_behav_folder, "Trial_data.mat")
    AllTrialsPath = AllTrialsPath.replace(r"D:\Alice\AK_data", r"E:\AK_data") # AllTrils.mat where pulse onsets are (after merge of left and right PC)

    # dt = update_dt_with_matlab_pulse_onsets(nc_path, AllTrialsPath)
    dt_accel = dt.copy()  
    def keep_pulse_onsets(ds):
        keep = {k: ds[k] for k in ds.data_vars if k == "pulse_onsets"}
        return xr.Dataset(keep, attrs=ds.attrs)
    dt_accel = dt_accel.map_over_datasets(keep_pulse_onsets)
    dt_accel = TrialTree(dt_accel)
        

    
    session = dt.itrial(0).session
    session = session.replace("-", "")
    bird = dt.itrial(0).bird
    
    dat_path = Path(accel_path) / bird / session / "auxiliary.dat"
    time_path = Path(accel_path) / bird / session / "time.dat"
    
    aux = read_intan_auxiliary(dat_path)
    time = read_intan_timestamps(time_path)
    
    for trial in dt.trials:
        time_frames = dt.trial(trial).time.values
        
        
        
        
        pulse_onsets = dt.trial(trial).pulse_onsets.values
    

    
        trial_sample_indices = np.arange(pulse_onsets[0], pulse_onsets[-1] + 150)
        
        aux_trial = aux[trial_sample_indices, :]
        time_trial = time[trial_sample_indices] - time[trial_sample_indices[0]]

        # aux_accel, aux_velocity, aux_speed = compute_aux_velocity_and_speed(
        #     aux_trial, time_trial, 
        #     mov_mean_window1=6001, 
        #     mov_mean_window2=15001
        # )

        # ds = dt[f"trial_{trial}"].to_dataset().copy()        


        # # As model feature
        # time_frames = dt.trial(trial).time.values
        # aux_acceleration = resample_to_frames(aux_accel, time_trial, time_frames)
        # aux_acceleration = clip_by_percentiles(aux_acceleration, (5, 95))
        # aux_velocity = resample_to_frames(aux_velocity, time_trial, time_frames)
        # aux_speed = resample_to_frames(aux_speed, time_trial, time_frames)        
        
        # ds['aux_acceleration'] = xr.DataArray(
        #     aux_acceleration,
        #     dims=['time', 'space'],
        #     coords={'time': time_frames, 'space': ['x', 'y', 'z']}
        # )
        # ds['aux_velocity'] = xr.DataArray(
        #     aux_velocity,
        #     dims=['time', 'space'],
        #     coords={'time': time_frames, 'space': ['x', 'y', 'z']}
        # )
        # ds['aux_speed'] = xr.DataArray(
        #     aux_speed,
        #     dims=['time'],
        #     coords={'time': time_frames}
        # )
        # for feat in ["aux_acceleration", "aux_velocity", "aux_speed"]:
        #     ds[feat].attrs["type"] = "features"    
        
        # dt[f"trial_{trial}"] = ds
        
        


    downsample_factor = 10  # 30000 -> 3000 Hz
    accel_downsampled = downsample_with_antialiasing(aux_trial, downsample_factor)
 
    time_downsampled = time_trial[::downsample_factor][:len(accel_downsampled)]

    for trial in dt_accel:
        ds = dt[trial].to_dataset().copy()   
        
        ds['acceleration'] = xr.DataArray(
            accel_downsampled,
            dims=['time_1000Hz', 'space'],
            coords={'time_1000Hz': time_downsampled, 'space': ['x', 'y', 'z']},
            attrs={'type': 'features'},
        )
        dt_accel[trial] = ds           
    
    # dt.save()
    dt_accel.save(nc_path.with_stem("Trial_data_accelerometer"))